This notebook demonstrates how use the model api to run predictions and get information from the model. You can use it as basis to write your own experiements using the model in this container.

To start, import _modelapi.model_, which will give us access to the api of the model in this container.

In [ ]:
from modelapi import model

In [ ]:
# other helpful imports for this demo
import h5py
import json
import matplotlib.pyplot as plt
from dipy.io.image import load_nifti, save_nifti

##### Various ways of handing data to the model

In [ ]:
# You can either load the necessary input for the prediction from a file:
input = 'sample_data/sample.json'
# or assemble a dictionary for yourself 
input_dict = {"t1":{},"t2":{},"flair":{},"t1c":{}}
# mandatory: fileurl for each input!
input_dict["t1"]["fileurl"] = 'sample_data/patient_1/t1.nii.gz'
# mandatory: format for each input, be aware that this has to be a list:
input_dict["t1"]["format"] = ["application/nii-gzip"]
# and so on for each other input.. 

In [ ]:
with open(input, "r") as f:
    input_dict_from_json = json.load(f)

In [ ]:
# you can then simply pass your input to the model:
result = model.predict(input_dict_from_json)

In [ ]:
# let's see what the output contains:
result

##### Recover a prediction from the h5 file

In [ ]:
# recover prediction from h5 file
resultFile = result["output"][0]["prediction"].replace("api", "")
f = h5py.File(resultFile, 'r')
print(list(f.keys()))
# load the actual prediction and check for correct shape
prediction = f["Segmentation"]
print('Shape of prediction is: {}'.format(prediction.shape))

In [ ]:
# just plot one slice to see if it returns a sensible result 
plt.imshow(prediction[90], aspect=0.5)
print(prediction[90].max(), prediction[90].min())

##### Saving the result as a Nifti-1 file so we can use it

In [ ]:
# first, we load the metadata from one of the input images to populate the header:
affine = load_nifti('sample_data/patient_1/flair.nii.gz')[1]
save_nifti('sample_data/patient_1/segmentation.nii.gz', prediction, affine)

Ask the model for a list of sample data, then run a prediction on the first of the sample dataset, and then display the prediction result output.

If the model does not provide any sample data, print an error message (this part also shows how to read meta info - like the model's name - from the models config).

In [ ]:
samples = model.get_samples()
if len(samples["files"]) > 0:
    inputFile = samples["folder"] + "/" + samples["files"][0]
    result = model.predict(inputFile)
    for i, output in enumerate(result["output"]):
        print("\n" + result["model"]["name"] + " Output " + str(i) + ":")
        print("  Type:  " + output["type"])
        print("  Name:  " + output["name"])
        print("  Shape: " + str(output["shape"]))
        print("  Prediction:")
        if output["type"] in ["mask_image", "image"]:
            # if output seems to be an image, try displaying it nicely with PIL
            try:
                import PIL
                image = PIL.Image.fromarray(output["prediction"])
                display(image)
            except:
                display(output["prediction"])
        else:
            display(output["prediction"])
else:
    model_name = model.get_config()["meta"]["name"]
    print(model_name + " does not provide any sample data, please try your own data.")